The 3 answers to 3 questions can be found at the end of each part.

In [1]:
#importing required libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

from sklearn.cluster import KMeans

print("Required Libraries have been imported.")

Required Libraries have been imported.


In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('Soup & CSV successfully imported.')

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

print('soup ready')

Soup & CSV successfully imported.
soup ready


In [3]:
table = soup.find('table', {'class':'wikitable sortable'})

In [4]:
tableRows = table.find_all('tr')

In [5]:
data = []
for row in tableRows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

print(df.head(5))
df.info()
df.shape

  PostalCode           Borough               Neighbourhood
1        M1A      Not assigned                            
2        M2A      Not assigned                            
3        M3A        North York                   Parkwoods
4        M4A        North York            Victoria Village
5        M5A  Downtown Toronto  Regent Park / Harbourfront
<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode       180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


(180, 3)

## PART 1: DATA CLEANING

In [6]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
print(df.shape)

df.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [7]:
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']

In [8]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [9]:
dfTor = df.groupby('PostalCode').agg(lambda x: ','.join(x))
dfTor.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [10]:
dfToronto = dfTor.reset_index()
dfToronto['Borough']= dfToronto['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [11]:
print(dfToronto.shape)
dfToronto.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
dfToronto.shape

(103, 3)

In [13]:
#installing Geopy & Geocoder
!pip install geopy
!pip install geocoder
print("Geopy & Geocoder has been installed")

Geopy & Geocoder has been installed


In [14]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
neighborhood = "Toronto, North York, Parkwoods"
loc = geolocator.geocode(neighborhood)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


latitude is :- 43.7587999 
longtitude is:- -79.3201966


In [15]:
dfGeopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

In [16]:
dfGeopy = dfToronto
dfGeopy['Neighbourhood'] = dfToronto[['Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
print(dfGeopy.shape)
dfGeopy

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [17]:
#Changing '/' with ',' as it is desired
dfGeopy['Neighbourhood'] = dfGeopy['Neighbourhood'].str.replace('/',',')
dfGeopy

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park"
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge"
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


## PART 2: ADDING GEOSPATIAL DATA TO THE DATAFRAME

In [18]:
dfGeopy['Latitude'] = '0';
dfGeopy['Longitude'] = '0';

# Reading the geospatial data from the url

dfGeo = pd.read_csv('https://cocl.us/Geospatial_data')
dfGeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
# Merging the latitude and longitude data into our main dataframe

for i in dfGeopy.index:
    for j in dfGeo.index:
        if dfGeopy.iloc[i,0] == dfGeo.iloc[j,0]:
            dfGeopy.iloc[i, 3] = dfGeo.iloc[j, 1]
            dfGeopy.iloc[i ,4] = dfGeo.iloc[j, 2]
            
dfGeopy.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


## PART 3: CLUSTERING & VISUALIZING THE NEIGHBORHOODS IN TORONTO

In [20]:
dfFinal = dfGeopy.copy()
dfFinal = dfFinal[dfGeopy['Borough'].str.contains("Toronto")]
dfFinal.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,EastToronto,The Beaches,43.6764,-79.293
41,M4K,EastToronto,"The Danforth West , Riverdale",43.6796,-79.3522
42,M4L,EastToronto,"India Bazaar , The Beaches West",43.669,-79.3156
43,M4M,EastToronto,Studio District,43.6595,-79.3409
44,M4N,CentralToronto,Lawrence Park,43.728,-79.3888


In [21]:
#First thing to create a Toronto map is to get coordinates of Toronto

loc = geolocator.geocode('Toronto')
print("Toronto's latitude is :-" ,loc.latitude,"\nToronto's longtitude is:-" ,loc.longitude)

# By importing Folium, we can create a clustering map with the colors we defined

import folium

mapToronto = folium.Map(location = [loc.latitude, loc.longitude], zoom_start = 11.9)

X = dfFinal['Latitude']
Y = dfFinal['Longitude']
Z = np.stack((X,Y), axis = 1)

kmeans = KMeans(n_clusters = 4, random_state = 0).fit(Z)

clusters = kmeans.labels_
colors = ['orange', 'purple', 'green', 'yellow']
dfFinal['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(dfFinal['Latitude'], dfFinal['Longitude'], dfFinal['Borough'], dfFinal['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(mapToronto) 

mapToronto

Toronto's latitude is :- 43.6534817 
Toronto's longtitude is:- -79.3839347
